<a href="https://colab.research.google.com/github/sarth-04/GenAI-for-Research-and-Education/blob/main/Gen_AI_for_Research_and_Education.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all the required libraries
!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers langgraph langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.7 MB/s eta 0:

In [2]:
!pip install langchain

In [3]:
!pip install langgraph

In [4]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00


In [5]:
from typing import Literal
from pathlib import Path
import pandas as pd
import pdfplumber
from operator import itemgetter
import json
import openai
import chromadb
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langchain.memory import ConversationBufferMemory
from langchain_core.messages import RemoveMessage
from langgraph.checkpoint.memory import MemorySaver

In [6]:
# Initialize the memory for the checkpointer
memory = MemorySaver()

In [8]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [9]:
# Define the path to the directory containing the PDF files
pdf_path = "/content/drive/MyDrive/LLM"


In [10]:
# Function to check if a word is inside a table bbox
def check_bboxes(word, table_bbox):
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

In [11]:
# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    full_text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {len(full_text) + 1}"
            text = page.extract_text()
            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []
            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):
                # Check if the item in the cluster is a word dictionary or a table dictionary
                if 'text' in cluster[0]:  # Word dictionary
                    lines.append(' '.join([i['text'] for i in cluster if 'text' in i]))
                elif 'table' in cluster[0]:  # Table dictionary
                    lines.append(json.dumps(cluster[0]['table']))
            full_text.append([page_no, " ".join(lines)])
    return full_text

In [12]:
# Define the directory containing the PDF files
pdf_directory = Path(pdf_path)

In [13]:
# Initialize an empty list to store the extracted texts and document names
data = []

In [14]:
# Loop through all files in the directory and extract text
for pdf_path in pdf_directory.glob("*.pdf"):
    print(f"...Processing {pdf_path.name}")
    extracted_text = extract_text_from_pdf(pdf_path)
    extracted_text_df = pd.DataFrame(extracted_text, columns=['Page No.', 'Page_Text'])
    extracted_text_df['Document Name'] = pdf_path.name
    data.append(extracted_text_df)
    print(f"Finished processing {pdf_path.name}")


...Processing 00.DSA_-_CP_Revision.pdf
Finished processing 00.DSA_-_CP_Revision.pdf


In [15]:

print("All PDFs have been processed.")
gk_pdfs_data = pd.concat(data, ignore_index=True)

All PDFs have been processed.


In [16]:

# Clean and filter data
gk_pdfs_data['Text_Length'] = gk_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))
gk_pdfs_data = gk_pdfs_data.loc[gk_pdfs_data['Text_Length'] >= 10]

In [17]:
# Add metadata
gk_pdfs_data['Metadata'] = gk_pdfs_data.apply(lambda x: {'Document_Name': x['Document Name'][:-4], 'Page_No.': x['Page No.']}, axis=1)

In [18]:
# Set OpenAI API key for LangChain use
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
openai.api_key = os.environ["OPENAI_API_KEY"]

OPENAI_API_KEY: ··········


In [19]:
# Set up ChromaDB and embedding function
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
client = chromadb.PersistentClient()
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name="text-embedding-ada-002")

In [20]:
# Create or retrieve ChromaDB collection
gk_collection = client.get_or_create_collection(name="gk_collection", embedding_function=embedding_function)

In [21]:
# Add the extracted page text and metadata to the ChromaDB collection
documents_list = gk_pdfs_data["Page_Text"].tolist()
metadata_list = gk_pdfs_data['Metadata'].tolist()
gk_collection.add(
    documents=documents_list,
    ids=[str(i) for i in range(len(documents_list))],
    metadatas=metadata_list
)


TypeError: APIStatusError.__init__() missing 2 required keyword-only arguments: 'response' and 'body'

In [ ]:
# Custom retrieval function using ChromaDB
def chroma_retriever(query: str, collection: chromadb.api.models.Collection, n_results: int = 10):
    results = collection.query(query_texts=[query], n_results=n_results)
    documents = results['documents'][0] if 'documents' in results else []
    metadatas = results['metadatas'][0] if 'metadatas' in results else []
    return documents, metadatas

# Initialize Chat Model (OpenAI GPT)
chat_model = ChatOpenAI(model_name="gpt-4o-mini")

# Retrieval tool definition using the retriever
from langchain.tools import Tool

retriever_tool = Tool.from_function(
    func=lambda query: chroma_retriever(query, gk_collection),
    name="retrieve_pdfs",
    description="Retrieve relevant information from PDF documents related to antenatal wellness."
)

# Define a custom state for summarization and conversation
class State(MessagesState):
    summary: str

# Summarization function
def summarize_conversation(state: State):
    summary_message = "Summarize the following conversation: " + state["messages"][0].content
    response = chat_model([HumanMessage(content=summary_message)])
    return {"summary": response.content}

# Conditional logic to determine when to summarize
def should_continue(state: State) -> Literal["summarize_conversation", "retrieve", END]: # Modified to include 'retrieve'
    if len(state["messages"]) > 6:
        return "summarize_conversation"
    else:
        return "retrieve" # Added to trigger retrieval after conversation
    # return END # Removed to prevent premature ending


# Agent logic to process input using the chat model
def agent(state: MessagesState):
    response = chat_model([SystemMessage(content="Handle user query.")]+state["messages"])
    return {"messages": [response]}

# Document retrieval node
def retrieve(state: MessagesState):
    query = state["messages"][-1].content
    documents, metadatas = retriever_tool(query)
    retrieved_messages = [AIMessage(content=f"Document: {doc}\nMetadata: {meta}") for doc, meta in zip(documents, metadatas)]
    return {"messages": retrieved_messages}

# Workflow definition
workflow = StateGraph(State)
workflow.add_node("conversation", agent)
workflow.add_node("retrieve", retrieve)
workflow.add_node("summarize_conversation", summarize_conversation)

# Adding workflow edges and logic
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue) # Now includes 'retrieve' as a possible transition
workflow.add_edge("summarize_conversation", END)
# workflow.add_edge("retrieve", END) # Removed, as 'retrieve' is now conditionally called
workflow.add_edge("retrieve", "conversation") # Added to create a loop back to conversation after retrieval


# Compiling the workflow with a memory checkpointer
app = workflow.compile(checkpointer=memory)

In [ ]:
# Import the Image class from IPython.display
from IPython.display import Image
from langchain.schema import HumanMessage, SystemMessage
# Show
display(Image(app.get_graph(xray=True).draw_mermaid_png()))